# Lab 03: Lagrangian Relaxation

Made & Presented by Bo Tang

In this lab, you will explore **Lagrangian Relaxation** with Gurobi and Python. You will gain hands-on experience solving complex optimization problems by relaxing difficult constraints, adjusting Lagrange multipliers iteratively, and solving the resulting relaxed subproblems.

In [ ]:
# install gurobipy first
! pip install gurobipy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 29.1 MB/s eta 0:00:00


In [ ]:
import gurobipy as gp
import numpy as np
from gurobipy import GRB

### Lagrangian Relaxation Review

Lagrangian Relaxation is an optimization technique used to solve complex constrained problems by relaxing certain constraints and incorporating them into the objective function using Lagrange multipliers. This technique is particularly useful when some constraints make a problem hard to solve directly. By relaxing the constraints, we can decompose the problem into easier subproblems and iteratively adjust the Lagrange multipliers to converge to an optimal solution.

Consider the following linear optimization problem, where we aim to minimize the objective function:

$$
\min_x \mathbf{c}^T \mathbf{x}
$$
Subject to
$$
\mathbf{A} \mathbf{x} \leq \mathbf{b} \\
\mathbf{D} \mathbf{x} \leq \mathbf{d} \\
\mathbf{x} \geq \mathbf{0}
$$

If the constraints $\mathbf{A} \mathbf{x} \leq \mathbf{b}$ are complicated, we can relax them by removing the constraints and introducing a **penalty term** into the objective function. This penalty term is weighted by a vector of Lagrange multipliers, $\mathbf{\lambda} \geq \mathbf{0}$, which penalizes violations of the relaxed constraints. This process is known as **Lagrangian Relaxation**.

$$
\min_x \mathbf{c}^T \mathbf{x} + \lambda^T (\mathbf{A} \mathbf{x} - \mathbf{b})
$$
Subject to
$$
\mathbf{D} \mathbf{x} \leq \mathbf{d} \\
\mathbf{x} \geq \mathbf{0}
$$

This relaxed problem is generally easier to solve than the original problem because the complex constraints $\mathbf{A} \mathbf{x} \leq \mathbf{b}$ have been removed. However, for any given value of the Lagrange multipliers $\mathbf{\lambda} \geq \mathbf{0}$, the solution to this relaxed problem only provides a lower bound on the objective value of orginal problem.

By removing constant term $- \mathbf{\lambda}^T \mathbf{b}$, the simplified objective function becomes $(\mathbf{c}^T + \lambda^T \mathbf{A}) \mathbf{x}$.


**Question:**
- What happens in a maximization problem?
- What if the constraints are $\mathbf{A} \mathbf{x} \geq \mathbf{b}$ or $\mathbf{A} \mathbf{x} = \mathbf{b}$?

#### Iterative Process

The iterative process involves solving the this problem as subprolem, which seeks to maximize the lower bound obtained from the relaxed problem. This can be done by optimizing the Lagrange multipliers $\mathbf{\lambda}$.

Algorithm:

1. Initialize first feasible solution $\mathbf{x}^0$ and master problem:
$$
\max_{\mathbf{\lambda} \geq \mathbf{0}, z} z
$$
2. Generate new constraint to the master with the given $\mathbf{x}^k$:
$$
z \leq \mathbf{c}^T \mathbf{x}^k + \lambda^T (\mathbf{A} \mathbf{x}^k - \mathbf{b})
$$
3. Solve the new master prolem. This provides the lower bound:
$$z^k$$
Terminate if the gap is 0.
4. Solve the subproblems (Lagrangian Relaxation) with the given $\mathbf{\lambda}^k$:
$$
\min_x (\mathbf{c}^T + {\lambda^k}^T \mathbf{A}) \mathbf{x}
$$
Subject to
$$
\mathbf{D} \mathbf{x} \leq \mathbf{d} \\
\mathbf{x} \geq \mathbf{0}
$$
This provides the lower bound:
$$
\mathbf{c}^T \mathbf{x}^{k+1} + {\mathbf{\lambda}^k}^T (\mathbf{A} \mathbf{x}^{k+1} - \mathbf{b}).
$$
If the gap between the upper and lower bounds is zero, terminate.
5. Return to step 2.


### Example 1: Pike and Quid Problem from Note 3b

We now apply the Lagrangian Relaxation method to solve the Pike and Quid Problem. In this problem, we want to maximize the steel production across two locations, Pike and Quid, while adhering to resource constraints.

#### Problem Parameters:

Objective coefficients $\mathbf{c}$ representing the production value of different types of steel produced at Pike and Quid.

In [ ]:
c = np.array([90, 80, 70, 60])

Total ore constraint (first constraint):
- $\mathbf{A}$ represents the coefficients of the total ore consumption for different types of steel.
- $\mathbf{b}$, representing the total available ore supply.

In [ ]:
A = np.array([[8, 6, 7, 5]])
b = np.array([80])

Resource constraints at Pike and Quid (coal and furnace capacities):
- $\mathbf{D}$​​ representing the resource consumption constraints for coal and furnace at both locations.
- $\mathbf{d}$​​  representing the maximum allowable resources (coal and furnace) at Pike and Quid.

In [ ]:
D = np.array([[3, 1, 0, 0],
              [2, 1, 0, 0],
              [0, 0, 3, 2],
              [0, 0, 1, 1]])
d = np.array([12, 10, 15, 4])

#### Original LP

We first solve the problem with original formulation. Before implementing any fancy algorithms like **Lagrangian Relaxation**, it is essential to first solve the problem using conventional methods to establish a **baseline** and **solution check**.

In [ ]:
# init model
model = gp.Model("Orignal Production")

# decision variables
x = model.addMVar(4, name="steels")

# objective function
model.setObjective(c @ x, sense=GRB.MAXIMIZE)

# constraints
model.addConstr(A @ x <= b)
model.addConstr(D @ x <= d)

# solves
model.optimize()

# solution
print("Objective Value: {:.2f}".format(model.ObjVal))
print(f"{x[0].x:5.2f} tons Steel1 from Pike.")
print(f"{x[1].x:5.2f} tons Steel2 from Pike.")
print(f"{x[2].x:5.2f} tons Steel1 from Quid.")
print(f"{x[3].x:5.2f} tons Steel2 from Quid.")
print("\n\n")

**Understanding MVar in Gurobi:*** In Gurobi, an MVar (matrix variable) is used to handle multiple decision variables at once, especially when the decision variables are naturally represented as a vector or matrix (e.g., when there are multiple production decisions to make). Using MVar allows you to directly perform matrix operations `@` in the objective function and constraints.

#### Task: Complete Iterative Algorithm

Now that we have established a baseline solution using the traditional method, we can proceed with implementing the **Lagrangian Relaxation** algorithm to iteratively solve the problem.

Please fill the empty code block.

##### Initialize First Feasible Solution

In [ ]:
xval = np.zeros_like(c)

##### Master Problem

In [ ]:
# init master problem
master = gp.Model("Master Problem")
# turn off log
master.Params.outputFlag = 0
# decision variables
λ = master.addMVar(len(A), name="dual")
z = master.addVar(name="z")
# objective function
master.setObjective(z, sense=GRB.MINIMIZE)

##### Lagragian Subproblem

In [ ]:
# init subproblem
subproblem = gp.Model("Subproblem")
# turn off log
subproblem.Params.outputFlag = 0
# TODO: decision variables and constraints

##### Iterations

In [ ]:
ub = np.inf
# iterative updates
cnt = 0
while True:
    # count
    cnt +=  1
    # TODO: add new constraint master
    master.addConstr
    # solve master for λ
    master.optimize()
    λval = λ.X
    # TODO: lower bound
    lb =
    # terminate condition
    if ub - lb < 1e-6:
        break
    # TODO: update subproblem obj
    subproblem.setObjective
    # solve the subproblems for x
    subproblem.optimize()
    xval = x.X
    # TODO: upper bound
    ub =
    # terminate
    if ub - lb < 1e-6:
        break
    print(f"Iteration {cnt-1}:")
    print(f"  λ = {λval.tolist()}, Dual Obj = {lb:.2f}.")
    print(f"  x = {xval.tolist()}, Primal Obj = {ub:.2f}.\n")
print(f"Iteration {cnt-1}:")
print(f"  λ = {λval.tolist()}, Dual Obj = {lb:.2f}.")
print(f"  x = {xval.tolist()}, Primal Obj = {ub:.2f}.\n")

##### Solution Output

In [ ]:
# solution
print("Objective Value: {:.2f}".format(model.ObjVal))
print(f"{x[0].x:5.2f} tons Steel1 from Pike.")
print(f"{x[1].x:5.2f} tons Steel2 from Pike.")
print(f"{x[2].x:5.2f} tons Steel1 from Quid.")
print(f"{x[3].x:5.2f} tons Steel2 from Quid.")
print("\n\n")

**Question:**

- What if we decompose subproblem for Pike and Quid?

In [ ]:
c1, c2 = c[:2], c[2:]
D1, D2 = D[:2,:2], D[2:,2:]
d1, d2 = d[:2], d[2:]

In [ ]:
# init subproblem 1
subproblem1 = gp.Model("Subproblem 1")
# turn off log
subproblem1.Params.outputFlag = 0
# decision variables
x1 = subproblem1.addMVar(len(c1), name="steels")
# TODO: constraints

In [ ]:
# init subproblem 2
subproblem2 = gp.Model("Subproblem 2")
# turn off log
subproblem2.Params.outputFlag = 0
# decision variables
x2 = subproblem2.addMVar(len(c2), name="steels")
# TODO: constraints

### Example 2: Logistics Distribution Problem

A company needs to deliver goods to three destinations (Destination 1, 2, 3) from two warehouses (Warehouse 1, 2). Each warehouse has a limited amount of goods available, and the vehicles used for transportation have limited capacities. Each destination has different demand levels, and the company wants to minimize the total transportation cost.

#### Data

##### 1. Transportation cost $c_{ij}$:
This represents the cost of transporting one unit of goods from warehouse $i$ to destination $j$.

| Warehouse/Destination  | Destination 1 | Destination 2 | Destination 3 |
|------------------------|---------------|---------------|---------------|
| **Warehouse 1**         | 2             | 4             | 5             |
| **Warehouse 2**         | 3             | 1             | 2             |

##### 2. Inventory at each warehouse $s_i$:
This is the maximum amount of goods available at each warehouse.

| Warehouse               | Inventory     |
|-------------------------|---------------|
| **Warehouse 1**          | 70            |
| **Warehouse 2**          | 90            |

##### 3. Demand at each destination $d_j$:
This is the amount of goods required at each destination.

| Destination             | Demand        |
|-------------------------|---------------|
| **Destination 1**        | 60            |
| **Destination 2**        | 50            |
| **Destination 3**        | 40            |

##### 4. Vehicle capacity $t_i$:
This is the maximum carrying capacity of vehicle for each warehouse.

| Vehicle                 | Capacity      |
|-------------------------|---------------|
| **Car for Warehouse 1** | 80            |
| **Car for Warehouse 2** | 100           |

#### Objective

The objective is to minimize the transportation cost while ensuring that all demand is satisfied, inventory limits are respected, and the vehicles' carrying capacity is not exceeded.

#### Lagrangian Relaxation

We will relax the **vehicle capacity constraint** and solve the problem iteratively.

#### Step 1: Formulate the Problem as an LP Model

Before applying Lagrangian Relaxation, it's essential to first formulate and solve the problem using traditional linear programming (LP) methods. This will provide us with a baseline solution to compare against future, more complex algorithms.

In [ ]:
# init model
model = gp.Model("Orignal Distribution")

#### Step 2: Applying Lagrangian Relaxation

We will now attempt to solve the problem using Lagrangian Relaxation. If you find this implementation challenging, you can consider solving it step by step manually, i.e., solving the subproblems with Gurobi, then copying and pasting the results back into the master problem, and vice versa.

**Question:**
- What if we relaxed other constraints instead?